# Question 1

In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium
import geocoder

In [15]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [16]:
soup = BeautifulSoup(page.content, 'html.parser')

In [17]:
table=soup.find('table')

In [18]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':#
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [19]:
df=pd.DataFrame(table_contents)

In [20]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

check if any PostalCode is used multiple times

In [21]:
df['Postal Code'].is_unique

True

Check if for any entry Neighborhood is not assigned

In [22]:
'Not assigned' in df['Neighborhood'].values

False

In [23]:
df.shape

(103, 3)

# Question 2

In [25]:
# Get latitude and longitude and add them to dataframe
lat_lng_coords = None
lat = []
lon = []
for i in range(df.shape[0]):
    while(lat_lng_coords is None):
        g = geocoder.google(df.loc[i, 'Borough'] + ', {}'.format(df.loc[i, 'Postal Code']))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lat.append(latitude)
    lon.append(longitude)
df['Latitude'] = lat
df['Longitude'] = lon

KeyboardInterrupt: 

#### I tries several times but geocoder is not working

Thatswhy I have used the given csv file

In [26]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [27]:
df_coordinates = pd.merge(df, coordinates, on="Postal Code")

In [28]:
df_coordinates.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Question 3

## Didn't understand the question exactly

In [29]:
map_toronto = folium.Map(location=[43.753259, -79.329656], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coordinates['Latitude'], df_coordinates['Longitude'], df_coordinates['Borough'], df_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto